# Data Augemntation - Image Manipulation 
1. Geometric Transformations (Rotations, Translation, Shearing, Flipping)
2. Non- Geometric Transformations (Cropping, Noise injection, Color Space, Jitter, Kernel)

In [1]:
import torch
import shutil
import torchvision
from torchvision import models,datasets,transforms
import os
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import cv2
import random
import albumentations as A #this supposedly is much faster than using torchvision
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

C:\Users\vidia\AppData\Roaming\Python\Python312\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.4' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
#create augmentation pipeline
#all of these need to be tuned for now these are some safe choices

augmentation_pipeline = A.Compose([
    # Geometric Transformations
    A.Rotate(limit=25, p=0.5),  # rotation +-25 with prob 0.5
    A.ShiftScaleRotate(shift_limit=0.3, scale_limit=0.2, rotate_limit=0, p=0.3),  # translation (shift and scale but do not rotate more)
    A.Affine(shear=5, p=0.5),  # shearing up to 5deg - does not seem that useful for us
    A.HorizontalFlip(p=0.5),  # flipping h

    # Non-Geometric Transformations
    A.RandomResizedCrop(size=[524,524], scale=(0.7, 1.0), p=0.5),  # gets randomly 70-100% of image and resized it back to 256x256
    A.GaussNoise(var_limit=(2, 10), p=0.2),  # less variance
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.2), # contrast for color (insetad of b&w)
    A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05, p=0.3),  # Color Adjustments different lightning conditions
    A.GaussianBlur(blur_limit=(3, 7), p=0.2),  # Kernel Blurring

    A.Resize(524, 524),  # resizing to 524x524 (then using tensor pipeline will be 256x256)

    ToTensorV2() #this computes it back to tensor - to use for pytorch
])

C:\Users\vidia\AppData\Roaming\Python\Python312\site-packages\albumentations\core\validation.py:58: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\vidia\AppData\Local\Temp\ipykernel_11508\1580372755.py:13: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(2, 10), p=0.2),  # less variance


In [5]:
# use the pipeline to also balance the dataset
# idea : augment images until limit set as majority_class*2

# if you want to test for yourself then update directories to "../example_data/dataset_balanced/train"
output_dir = "../data/dataset_balanced/train"


# define the paths
input_dir = "../data/dataset_split/train"  
output_dir = "../data/dataset_balanced/train"  # for balanced augmented dataset

# overwrite it
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)  # delete contents

os.makedirs(output_dir, exist_ok=True)

class_counts = {}
for class_folder in os.listdir(input_dir):
    class_path = os.path.join(input_dir, class_folder)
    num_images = len(os.listdir(class_path))
    class_counts[class_folder] = num_images

# change this
change_factor = 2

# Find the largest class (majority class)
max_class_size = max(class_counts.values())
new_target_size = max_class_size * change_factor

print(class_counts)
print(f"max category is {max_class_size} of class {max(class_counts, key=class_counts.get)}") 

for class_folder, current_count in tqdm(class_counts.items(), desc="Balancing & Expanding Classes"):
    class_path = os.path.join(input_dir, class_folder)
    augmented_class_path = os.path.join(output_dir, class_folder) #augmented images will be saved here
    os.makedirs(augmented_class_path, exist_ok=True)

    images = os.listdir(class_path)
    
    # decide later but this will copy all the non-augmented images as well 
    for img_name in images:
        src_path = os.path.join(class_path, img_name)
        dst_path = os.path.join(augmented_class_path, img_name)
        cv2.imwrite(dst_path, cv2.imread(src_path))  # Copy image

    # Compute number of extra images needed
    num_needed = new_target_size - current_count  # balancing all classes to 2*majority

    # Augment existing images
    while num_needed > 0:
        for img_name in images:
            if num_needed <= 0:
                break 

            img_path = os.path.join(class_path, img_name)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR (OpenCV) to RGB

            augmented = augmentation_pipeline(image=image)["image"]  # Albumentations output

            # Save Augmented Image (Directly as PNG)
            output_filename = f"{os.path.splitext(img_name)[0]}_aug_{num_needed}.png"
            output_path = os.path.join(augmented_class_path, output_filename)

            # Ensure the output is a NumPy array before saving
            if isinstance(augmented, torch.Tensor):  
                augmented = augmented.permute(1, 2, 0).cpu().numpy()  # Convert CHW -> HWC
                augmented = (augmented * 255).astype(np.uint8)  # Convert from [0,1] to [0,255]

            # Save the augmented image
            cv2.imwrite(output_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))
            num_needed -= 1

print(f"✅ Dataset balanced & expanded! New images saved in {output_dir}")

{'cardboard': 368, 'glass': 336, 'metal': 632, 'paper': 400, 'plastic': 736, 'trash': 396}
max category is 736 of class plastic


Balancing & Expanding Classes: 100%|██████████| 6/6 [01:49<00:00, 18.26s/it]

✅ Dataset balanced & expanded! New images saved in ../data/dataset_balanced/train


In [32]:
#sos remember to change the directory for the tensors
dir = '../data/dataset_balanced'

params = { 'batch_size':16,
           'shuffle':True,
           'num_workers':4 }


transform = transforms.Compose([transforms.Resize(256),
                                transforms.RandomResizedCrop(256), #Augmented
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#the training dataset will be from the datase_balanced folder 
train_dataset = datasets.ImageFolder(os.path.join(dir, 'train'),transform = transform )

transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])


#change the dir for validationa (and test)
dir = '../data/dataset_split'
val_dataset = datasets.ImageFolder(os.path.join(dir, 'val'),transform = transform )


#data loaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, **params)
val_dataloader = torch.utils.data.DataLoader(val_dataset, **params)

class_names = train_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")